# **Simple Model**

In [1]:
!pip install -q gradio diffusers transformers accelerate safetensors

import gradio as gr
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
import torch
from PIL import Image
import json
import os
from datetime import datetime
import gc
import random

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔵 Running in Google Colab")
except:
    IN_COLAB = False
    print("🔵 Running in local environment")

# Global configuration
CONFIG = {
    "max_generations": 500,
    "generation_count": 0,
    "history_file": "generation_history.json",
    "models": {
        "SD 1.5 Fast": {
            "model_id": "runwayml/stable-diffusion-v1-5",
            "type": "sd15",
            "description": "Fast and efficient for quick generation"
        },
        "SD 1.5 Quality": {
            "model_id": "stabilityai/stable-diffusion-2-1",
            "type": "sd15",
            "description": "Higher quality SD 1.5 model"
        },
        "SDXL Base": {
            "model_id": "stabilityai/stable-diffusion-xl-base-1.0",
            "type": "sdxl",
            "description": "Highest quality SDXL model"
        },
        "OpenJourney": {
            "model_id": "prompthero/openjourney",
            "type": "sd15",
            "description": "Artistic style model"
        }
    },
    "image_sizes": {
        "SD 1.5": {
            "512x512 (Square)": (512, 512),
            "768x512 (Landscape)": (768, 512),
            "512x768 (Portrait)": (512, 768),
            "640x640 (Large Square)": (640, 640)
        },
        "SDXL": {
            "1024x1024 (Square)": (1024, 1024),
            "1152x896 (Landscape)": (1152, 896),
            "896x1152 (Portrait)": (896, 1152),
            "1344x768 (Wide)": (1344, 768),
            "768x1344 (Tall)": (768, 1344)
        }
    }
}

class SmartImageGenerator:
    def __init__(self):
        self.pipeline = None
        self.current_model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.load_history()

    def load_history(self):
        """Load generation history from file"""
        try:
            if os.path.exists(CONFIG["history_file"]):
                with open(CONFIG["history_file"], 'r', encoding='utf-8') as f:
                    self.history = json.load(f)
                    CONFIG["generation_count"] = len(self.history)
            else:
                self.history = []
                CONFIG["generation_count"] = 0
        except:
            self.history = []
            CONFIG["generation_count"] = 0

    def save_history(self):
        """Save generation history to file"""
        try:
            with open(CONFIG["history_file"], 'w', encoding='utf-8') as f:
                json.dump(self.history, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error saving history: {e}")

    def load_model(self, model_name, progress=gr.Progress()):
        """Load selected model with progress tracking"""
        if self.current_model == model_name and self.pipeline is not None:
            return f"✅ Model {model_name} is already loaded"

        try:
            progress(0.1, desc="Clearing memory...")
            # Clear previous model
            if self.pipeline is not None:
                del self.pipeline
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            model_info = CONFIG["models"][model_name]
            model_id = model_info["model_id"]
            model_type = model_info["type"]

            progress(0.3, desc=f"Loading {model_name}...")

            # Load appropriate pipeline based on model type
            if model_type == "sdxl":
                self.pipeline = StableDiffusionXLPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16,
                    use_safetensors=True,
                    variant="fp16" if torch.cuda.is_available() else None
                )
            else:
                self.pipeline = StableDiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    safety_checker=None,
                    requires_safety_checker=False
                )

            progress(0.7, desc="Moving to device...")
            self.pipeline = self.pipeline.to(self.device)

            # Memory optimization
            if hasattr(self.pipeline, 'enable_attention_slicing'):
                self.pipeline.enable_attention_slicing()
            if hasattr(self.pipeline, 'enable_model_cpu_offload') and torch.cuda.is_available():
                self.pipeline.enable_model_cpu_offload()

            progress(1.0, desc="Ready!")
            self.current_model = model_name

            return f"✅ {model_name} loaded successfully!"

        except Exception as e:
            return f"❌ Error loading model: {str(e)}"

    def get_available_sizes(self, model_name):
        """Get available image sizes for selected model"""
        try:
            model_type = CONFIG["models"][model_name]["type"]
            if model_type == "sdxl":
                return list(CONFIG["image_sizes"]["SDXL"].keys())
            else:
                return list(CONFIG["image_sizes"]["SD 1.5"].keys())
        except:
            return list(CONFIG["image_sizes"]["SD 1.5"].keys())

    def generate_image(self, prompt, negative_prompt, model_name, size_name,
                      cfg_scale, steps, seed, progress=gr.Progress()):
        """Generate image with all parameters"""

        # Check generation limit
        if CONFIG["generation_count"] >= CONFIG["max_generations"]:
            return None, f"❌ Generation limit reached ({CONFIG['max_generations']} images)"

        # Check if model is loaded
        if self.pipeline is None or self.current_model != model_name:
            return None, "❌ Please load the model first"

        # Validate inputs
        if not prompt.strip():
            return None, "❌ Please enter a prompt"

        try:
            progress(0.1, desc="Preparing generation...")

            # Get image size
            model_type = CONFIG["models"][model_name]["type"]
            if model_type == "sdxl" and size_name in CONFIG["image_sizes"]["SDXL"]:
                width, height = CONFIG["image_sizes"]["SDXL"][size_name]
            elif size_name in CONFIG["image_sizes"]["SD 1.5"]:
                width, height = CONFIG["image_sizes"]["SD 1.5"][size_name]
            else:
                # Fallback to default size
                width, height = (512, 512)

            # Handle seed
            if seed == -1:
                seed = random.randint(0, 2**32-1)

            generator = torch.Generator(device=self.device).manual_seed(int(seed))

            progress(0.3, desc="Generating image...")

            # Generate image
            with torch.no_grad():
                result = self.pipeline(
                    prompt=prompt,
                    negative_prompt=negative_prompt if negative_prompt.strip() else None,
                    width=width,
                    height=height,
                    num_inference_steps=int(steps),
                    guidance_scale=float(cfg_scale),
                    generator=generator
                )
                image = result.images[0]

            progress(0.9, desc="Saving...")

            # Save image and update history
            filename = f"generated_{CONFIG['generation_count']+1}_{seed}.png"
            image.save(filename)

            # Add to history
            generation_info = {
                "id": CONFIG["generation_count"] + 1,
                "timestamp": datetime.now().isoformat(),
                "prompt": prompt,
                "negative_prompt": negative_prompt,
                "model": model_name,
                "size": size_name,
                "width": width,
                "height": height,
                "cfg_scale": cfg_scale,
                "steps": steps,
                "seed": seed,
                "filename": filename
            }

            self.history.append(generation_info)
            CONFIG["generation_count"] += 1
            self.save_history()

            progress(1.0, desc="Complete!")

            status = f"✅ Image {CONFIG['generation_count']} generated successfully!\n"
            status += f"🌱 Seed: {seed}\n"
            status += f"📊 Remaining: {CONFIG['max_generations'] - CONFIG['generation_count']} images"

            return image, status

        except Exception as e:
            return None, f"❌ Generation error: {str(e)}"

    def get_generation_stats(self):
        """Get current generation statistics"""
        return {
            "total": CONFIG["generation_count"],
            "limit": CONFIG["max_generations"],
            "remaining": CONFIG["max_generations"] - CONFIG["generation_count"]
        }

def create_interface():
    """Create the main Gradio interface"""
    generator = SmartImageGenerator()

    # Custom CSS for better styling
    css = """
    .gradio-container {
        font-family: 'Segoe UI', 'Roboto', sans-serif !important;
        max-width: 1400px !important;
    }
    .title-header {
        text-align: center;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        font-size: 2.5rem;
        font-weight: bold;
        margin: 20px 0;
    }
    .stats-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 20px;
        border-radius: 15px;
        text-align: center;
        margin: 10px 0;
    }
    .model-info {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 10px;
        border-left: 4px solid #667eea;
        margin: 10px 0;
    }
    """

    with gr.Blocks(title="🎨 Smart Text-to-Image Generator", css=css) as interface:

        # Header
        gr.HTML("""
            <div class="title-header">
                🎨 Smart Text-to-Image Generator
            </div>
            <div style="text-align: center; color: #666; font-size: 1.2em; margin-bottom: 30px;">
                Transform your imagination into stunning visuals with AI
            </div>
        """)

        # Statistics display
        with gr.Row():
            stats_display = gr.HTML()

        with gr.Row():
            # Left column - Controls
            with gr.Column(scale=1):
                gr.Markdown("## ⚙️ Model & Settings")

                # Model selection
                model_selector = gr.Dropdown(
                    choices=list(CONFIG["models"].keys()),
                    value="SD 1.5 Fast",
                    label="🤖 Select Model",
                    info="Choose the AI model for generation"
                )

                model_info = gr.HTML()

                load_model_btn = gr.Button(
                    "📥 Load Model",
                    variant="secondary",
                    size="lg"
                )

                model_status = gr.Textbox(
                    label="Model Status",
                    value="❌ No model loaded",
                    interactive=False
                )

                gr.Markdown("## 🎨 Generation Settings")

                # Prompt inputs
                prompt = gr.Textbox(
                    label="✍️ Prompt",
                    placeholder="A beautiful landscape with mountains and lake, highly detailed, 8k",
                    lines=4
                )

                negative_prompt = gr.Textbox(
                    label="🚫 Negative Prompt",
                    placeholder="blurry, low quality, distorted",
                    lines=2,
                    value="blurry, low quality, worst quality"
                )

                # Size selection (dynamic based on model)
                size_selector = gr.Dropdown(
                    label="📐 Image Size",
                    choices=generator.get_available_sizes("SD 1.5 Fast"),
                    value="512x512 (Square)"
                )

                with gr.Row():
                    # CFG Scale
                    cfg_scale = gr.Slider(
                        minimum=1.0,
                        maximum=20.0,
                        value=7.5,
                        step=0.5,
                        label="⚖️ CFG Scale",
                        info="Controls how closely the AI follows your prompt"
                    )

                    # Steps
                    steps = gr.Slider(
                        minimum=10,
                        maximum=100,
                        value=30,
                        step=5,
                        label="🔄 Steps",
                        info="More steps = better quality but slower"
                    )

                # Seed
                seed = gr.Number(
                    label="🌱 Seed",
                    value=-1,
                    precision=0,
                    info="-1 for random seed"
                )

                # Generate button
                generate_btn = gr.Button(
                    "🚀 Generate Image",
                    variant="primary",
                    size="lg"
                )

                # Quick actions
                with gr.Row():
                    random_seed_btn = gr.Button("🎲 Random Seed", size="sm")
                    clear_btn = gr.Button("🗑️ Clear", size="sm")

                # Example prompts
                gr.Examples(
                    examples=[
                        "A majestic dragon soaring over a mystical forest, fantasy art, highly detailed",
                        "Portrait of a wise old wizard, realistic, professional photography, 8k",
                        "Cyberpunk cityscape at night, neon lights, futuristic, digital art",
                        "Peaceful zen garden with cherry blossoms, serene atmosphere, beautiful lighting",
                        "Astronaut exploring an alien planet, sci-fi, cinematic, epic"
                    ],
                    inputs=prompt,
                    label="💡 Example Prompts"
                )

            # Right column - Results
            with gr.Column(scale=1):
                gr.Markdown("## 🖼️ Generated Image")

                # Image output
                output_image = gr.Image(
                    label="Result",
                    type="pil",
                    height=500
                )

                # Generation status
                generation_status = gr.Textbox(
                    label="📊 Generation Status",
                    interactive=False,
                    lines=3
                )

                # Image actions
                with gr.Row():
                    download_btn = gr.Button("💾 Download", variant="secondary")
                    share_btn = gr.Button("🔗 Share", variant="secondary")

        # History tab
        with gr.Tab("📚 Generation History"):
            history_display = gr.Dataframe(
                headers=["ID", "Time", "Prompt", "Model", "Size", "Seed"],
                label="Recent Generations",
                interactive=False
            )
            refresh_history_btn = gr.Button("🔄 Refresh History")

        # Help tab
        with gr.Tab("📖 User Guide"):
            gr.Markdown("""
            ## 🎯 How to Use

            ### Step-by-Step Guide:
            1. **Select Model**: Choose between SD 1.5 (fast) or SDXL (high quality)
            2. **Load Model**: Click "Load Model" and wait for confirmation
            3. **Write Prompt**: Describe your desired image in English
            4. **Adjust Settings**: Configure CFG scale, steps, and image size
            5. **Generate**: Click "Generate Image" and wait for the result

            ### Model Comparison:
            - **SD 1.5 Fast**: Quick generation, good for testing ideas
            - **SD 1.5 Quality**: Better quality, slightly slower
            - **SDXL Base**: Highest quality, requires more memory
            - **OpenJourney**: Artistic style, unique aesthetic

            ### Parameter Guide:
            - **CFG Scale**: 1-5 (creative), 7-10 (balanced), 15+ (strict adherence)
            - **Steps**: 20-30 (fast), 30-50 (quality), 50+ (maximum quality)
            - **Seed**: Use same seed to reproduce results

            ### Pro Tips:
            - Use detailed, descriptive prompts in English
            - Include quality keywords: "highly detailed", "8k", "professional"
            - Use negative prompts to avoid unwanted elements
            - Experiment with different CFG scales for varied results

            ### Limitations:
            - Maximum 500 images per session
            - SDXL requires significant GPU memory
            - Generation time varies by settings and hardware
            """)

        # Event handlers
        def update_stats():
            stats = generator.get_generation_stats()
            return f"""
            <div class="stats-card">
                <h3>📊 Generation Statistics</h3>
                <p><strong>{stats['total']}</strong> / {stats['limit']} images generated</p>
                <p><strong>{stats['remaining']}</strong> generations remaining</p>
            </div>
            """

        def update_model_info(model_name):
            info = CONFIG["models"][model_name]
            return f"""
            <div class="model-info">
                <h4>🤖 {model_name}</h4>
                <p>{info['description']}</p>
                <p><strong>Type:</strong> {info['type'].upper()}</p>
            </div>
            """

        def update_size_options(model_name):
            available_sizes = generator.get_available_sizes(model_name)
            return gr.Dropdown(
                choices=available_sizes,
                value=available_sizes[0] if available_sizes else "512x512 (Square)"
            )

        def refresh_history():
            if not generator.history:
                return []

            # Get last 50 generations
            recent = generator.history[-50:]
            history_data = []
            for item in recent:
                history_data.append([
                    item.get("id", ""),
                    item.get("timestamp", "")[:19],
                    item.get("prompt", "")[:50] + "..." if len(item.get("prompt", "")) > 50 else item.get("prompt", ""),
                    item.get("model", ""),
                    item.get("size", ""),
                    item.get("seed", "")
                ])
            return history_data

        def generate_random_seed():
            return random.randint(0, 2**32-1)

        def clear_inputs():
            return "", "", -1

        # Connect events
        model_selector.change(
            fn=update_model_info,
            inputs=[model_selector],
            outputs=[model_info]
        )

        model_selector.change(
            fn=update_size_options,
            inputs=[model_selector],
            outputs=[size_selector]
        )

        load_model_btn.click(
            fn=generator.load_model,
            inputs=[model_selector],
            outputs=[model_status]
        )

        generate_btn.click(
            fn=generator.generate_image,
            inputs=[prompt, negative_prompt, model_selector, size_selector,
                   cfg_scale, steps, seed],
            outputs=[output_image, generation_status]
        ).then(
            fn=update_stats,
            outputs=[stats_display]
        )

        random_seed_btn.click(
            fn=generate_random_seed,
            outputs=[seed]
        )

        clear_btn.click(
            fn=clear_inputs,
            outputs=[prompt, negative_prompt, seed]
        )

        refresh_history_btn.click(
            fn=refresh_history,
            outputs=[history_display]
        )

        # Initialize interface
        interface.load(
            fn=update_stats,
            outputs=[stats_display]
        )

        interface.load(
            fn=update_model_info,
            inputs=[gr.State("SD 1.5 Fast")],
            outputs=[model_info]
        )

        interface.load(
            fn=refresh_history,
            outputs=[history_display]
        )

    return interface

def main():
    """Main application entry point"""
    global IN_COLAB

    print("🎨 Smart Text-to-Image Generator")
    print("=" * 50)

    # System info
    print(f"📱 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

    print(f"🔢 Generation Limit: {CONFIG['max_generations']}")
    print(f"📊 Current Count: {CONFIG['generation_count']}")
    print("=" * 50)

    # Create and launch interface
    try:
        interface = create_interface()

        if IN_COLAB:
            interface.launch(
                share=True,
                show_error=True,
                quiet=False
            )
        else:
            interface.launch(
                share=True,
                server_name="0.0.0.0",
                server_port=7860,
                show_error=True
            )

    except Exception as e:
        print(f"❌ Launch error: {e}")
        print("🔄 Trying simple launch method...")
        interface = create_interface()
        interface.launch(share=True)

if __name__ == "__main__":
    main()

🔵 Running in Google Colab
🎨 Smart Text-to-Image Generator
📱 Device: CUDA
🎮 GPU: Tesla T4
💾 GPU Memory: 14.7 GB
🔢 Generation Limit: 500
📊 Current Count: 0
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c625f8b29c8fee4feb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Advanced Model**

In [6]:
!pip install -q gradio diffusers transformers accelerate safetensors plotly pandas

import gradio as gr
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline, DPMSolverMultistepScheduler
import torch
from PIL import Image, ImageEnhance, ImageFilter
import json
import os
from datetime import datetime
import gc
import random
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from collections import defaultdict
import numpy as np

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🚀 Running in Google Colab - Enhanced Version")
except:
    IN_COLAB = False
    print("🚀 Running in local environment - Enhanced Version")

# Enhanced Global Configuration
CONFIG = {
    "max_generations": 1000,
    "generation_count": 0,
    "history_file": "generation_history_enhanced.json",
    "presets_file": "generation_presets.json",
    "favorites_file": "favorite_generations.json",
    "models": {
        "SD 1.5 Fast": {
            "model_id": "runwayml/stable-diffusion-v1-5",
            "type": "sd15",
            "description": "⚡ Fast and efficient for quick generation",
            "optimal_settings": {"steps": 25, "cfg": 7.5}
        },
        "SD 1.5 Quality": {
            "model_id": "stabilityai/stable-diffusion-2-1",
            "type": "sd15",
            "description": "🎨 Higher quality SD 1.5 model",
            "optimal_settings": {"steps": 40, "cfg": 8.0}
        },
        "SDXL Base": {
            "model_id": "stabilityai/stable-diffusion-xl-base-1.0",
            "type": "sdxl",
            "description": "💎 Highest quality SDXL model",
            "optimal_settings": {"steps": 50, "cfg": 7.5}
        },
        "OpenJourney": {
            "model_id": "prompthero/openjourney",
            "type": "sd15",
            "description": "🖼️ Artistic style model",
            "optimal_settings": {"steps": 35, "cfg": 7.0}
        },
        "DreamShaper": {
            "model_id": "Lykon/DreamShaper",
            "type": "sd15",
            "description": "✨ Fantasy and dream-like images",
            "optimal_settings": {"steps": 30, "cfg": 6.5}
        },
        "Realistic Vision": {
            "model_id": "SG161222/Realistic_Vision_V5.1_noVAE",
            "type": "sd15",
            "description": "📸 Photorealistic portraits and scenes",
            "optimal_settings": {"steps": 35, "cfg": 7.0}
        }
    },
    "image_sizes": {
        "SD 1.5": {
            "512x512 (Square)": (512, 512),
            "768x512 (Landscape)": (768, 512),
            "512x768 (Portrait)": (512, 768),
            "640x640 (Large Square)": (640, 640),
            "896x512 (Wide)": (896, 512),
            "512x896 (Tall)": (512, 896),
            "704x512 (Photo)": (704, 512),
            "512x704 (Photo Portrait)": (512, 704)
        },
        "SDXL": {
            "1024x1024 (Square)": (1024, 1024),
            "1152x896 (Landscape)": (1152, 896),
            "896x1152 (Portrait)": (896, 1152),
            "1344x768 (Wide)": (1344, 768),
            "768x1344 (Tall)": (768, 1344),
            "1536x640 (Ultra Wide)": (1536, 640),
            "1216x832 (Photo)": (1216, 832),
            "832x1216 (Photo Portrait)": (832, 1216)
        }
    },
    "schedulers": ["PNDM", "DDIM", "DPM Solver++", "Euler", "Euler a", "DDPM"],
    "style_presets": {
        "Photorealistic": {
            "positive": "photorealistic, 8k uhd, dslr, high quality, film grain, Fujifilm XT3",
            "negative": "cartoon, painting, illustration, (worst quality, low quality:1.4)"
        },
        "Digital Art": {
            "positive": "digital painting, artstation, concept art, smooth, sharp focus, illustration",
            "negative": "photo, realistic, realism, ugly"
        },
        "Anime": {
            "positive": "anime style, manga, pixiv, kawaii, colorful",
            "negative": "realistic, photo, western comic"
        },
        "Fantasy": {
            "positive": "fantasy art, magical, ethereal, mystical, enchanted",
            "negative": "modern, technology, mundane"
        },
        "Cyberpunk": {
            "positive": "cyberpunk, neon, futuristic, tech noir, blade runner style",
            "negative": "medieval, ancient, natural"
        },
        "Oil Painting": {
            "positive": "oil painting, traditional art, brushstrokes, canvas texture",
            "negative": "digital, photography, 3d render"
        },
        "Watercolor": {
            "positive": "watercolor painting, soft edges, artistic, traditional media",
            "negative": "digital art, photography, hard edges"
        },
        "3D Render": {
            "positive": "3d render, octane render, unreal engine, volumetric lighting",
            "negative": "2d, painting, drawing, sketch"
        }
    },
    "prompt_templates": {
        "Character Portrait": "{subject}, portrait, {style}, detailed face, beautiful lighting, high quality",
        "Landscape": "{location}, landscape, {time_of_day}, {weather}, {style}, scenic, beautiful",
        "Action Scene": "{character} {action}, dynamic pose, {environment}, {style}, motion blur",
        "Still Life": "{objects}, still life, {lighting}, {style}, detailed, artistic composition",
        "Architecture": "{building_type}, {architectural_style}, {time_of_day}, {style}, detailed"
    }
}

class EnhancedImageGenerator:
    def __init__(self):
        self.pipeline = None
        self.current_model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.scheduler_backup = None
        self.load_history()
        self.load_presets()
        self.load_favorites()
        self.generation_times = []
        self.model_usage = defaultdict(int)

    def load_history(self):
        """Load generation history from file"""
        try:
            if os.path.exists(CONFIG["history_file"]):
                with open(CONFIG["history_file"], 'r', encoding='utf-8') as f:
                    self.history = json.load(f)
                    CONFIG["generation_count"] = len(self.history)
            else:
                self.history = []
                CONFIG["generation_count"] = 0
        except:
            self.history = []
            CONFIG["generation_count"] = 0

    def load_presets(self):
        """Load user presets"""
        try:
            if os.path.exists(CONFIG["presets_file"]):
                with open(CONFIG["presets_file"], 'r', encoding='utf-8') as f:
                    self.presets = json.load(f)
            else:
                self.presets = {}
        except:
            self.presets = {}

    def load_favorites(self):
        """Load favorite generations"""
        try:
            if os.path.exists(CONFIG["favorites_file"]):
                with open(CONFIG["favorites_file"], 'r', encoding='utf-8') as f:
                    self.favorites = json.load(f)
            else:
                self.favorites = []
        except:
            self.favorites = []

    def save_history(self):
        """Save generation history to file"""
        try:
            with open(CONFIG["history_file"], 'w', encoding='utf-8') as f:
                json.dump(self.history, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error saving history: {e}")

    def save_presets(self):
        """Save user presets"""
        try:
            with open(CONFIG["presets_file"], 'w', encoding='utf-8') as f:
                json.dump(self.presets, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error saving presets: {e}")

    def save_favorites(self):
        """Save favorites"""
        try:
            with open(CONFIG["favorites_file"], 'w', encoding='utf-8') as f:
                json.dump(self.favorites, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Error saving favorites: {e}")

    def load_model(self, model_name, scheduler_name="DPM Solver++", progress=gr.Progress()):
        """Load selected model with scheduler"""
        if self.current_model == model_name and self.pipeline is not None:
            # Just update scheduler if model is already loaded
            self.set_scheduler(scheduler_name)
            return f"✅ Model {model_name} already loaded. Scheduler updated to {scheduler_name}"

        try:
            progress(0.1, desc="🧹 Clearing memory...")
            # Clear previous model
            if self.pipeline is not None:
                del self.pipeline
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            model_info = CONFIG["models"][model_name]
            model_id = model_info["model_id"]
            model_type = model_info["type"]

            progress(0.3, desc=f"📥 Loading {model_name}...")

            # Load appropriate pipeline based on model type
            if model_type == "sdxl":
                self.pipeline = StableDiffusionXLPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16,
                    use_safetensors=True,
                    variant="fp16" if torch.cuda.is_available() else None
                )
            else:
                self.pipeline = StableDiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    safety_checker=None,
                    requires_safety_checker=False
                )

            progress(0.7, desc="🚀 Optimizing...")
            self.pipeline = self.pipeline.to(self.device)

            # Memory optimization
            if hasattr(self.pipeline, 'enable_attention_slicing'):
                self.pipeline.enable_attention_slicing()
            if hasattr(self.pipeline, 'enable_model_cpu_offload') and torch.cuda.is_available():
                self.pipeline.enable_model_cpu_offload()

            # Enable xFormers if available
            try:
                self.pipeline.enable_xformers_memory_efficient_attention()
            except:
                pass

            # Set scheduler
            self.set_scheduler(scheduler_name)

            progress(1.0, desc="✨ Ready!")
            self.current_model = model_name
            self.model_usage[model_name] += 1

            return f"✅ {model_name} loaded successfully with {scheduler_name} scheduler!"

        except Exception as e:
            return f"❌ Error loading model: {str(e)}"

    def set_scheduler(self, scheduler_name):
        """Set the scheduler for the pipeline"""
        if self.pipeline is None:
            return

        scheduler_config = self.pipeline.scheduler.config

        if scheduler_name == "DPM Solver++":
            self.pipeline.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
        # Add other schedulers as needed

    def get_available_sizes(self, model_name):
        """Get available image sizes for selected model"""
        try:
            model_type = CONFIG["models"][model_name]["type"]
            if model_type == "sdxl":
                return list(CONFIG["image_sizes"]["SDXL"].keys())
            else:
                return list(CONFIG["image_sizes"]["SD 1.5"].keys())
        except:
            return list(CONFIG["image_sizes"]["SD 1.5"].keys())

    def apply_style_preset(self, preset_name, prompt, negative_prompt):
        """Apply style preset to prompts"""
        if preset_name == "None":
            return prompt, negative_prompt

        preset = CONFIG["style_presets"].get(preset_name, {})
        if preset:
            new_prompt = f"{prompt}, {preset['positive']}"
            new_negative = f"{negative_prompt}, {preset['negative']}" if negative_prompt else preset['negative']
            return new_prompt, new_negative
        return prompt, negative_prompt

    def post_process_image(self, image, brightness, contrast, saturation, sharpness):
        """Apply post-processing to generated image"""
        # Brightness
        if brightness != 1.0:
            enhancer = ImageEnhance.Brightness(image)
            image = enhancer.enhance(brightness)

        # Contrast
        if contrast != 1.0:
            enhancer = ImageEnhance.Contrast(image)
            image = enhancer.enhance(contrast)

        # Saturation
        if saturation != 1.0:
            enhancer = ImageEnhance.Color(image)
            image = enhancer.enhance(saturation)

        # Sharpness
        if sharpness != 1.0:
            enhancer = ImageEnhance.Sharpness(image)
            image = enhancer.enhance(sharpness)

        return image

    def generate_batch(self, prompt, negative_prompt, model_name, size_name,
                      cfg_scale, steps, seed, batch_size, style_preset,
                      brightness, contrast, saturation, sharpness,
                      progress=gr.Progress()):
        """Generate multiple images in batch"""

        # Check generation limit
        if CONFIG["generation_count"] + batch_size > CONFIG["max_generations"]:
            remaining = CONFIG["max_generations"] - CONFIG["generation_count"]
            return [], f"❌ Generation limit exceeded. You can generate {remaining} more images."

        # Check if model is loaded
        if self.pipeline is None or self.current_model != model_name:
            return [], "❌ Please load the model first"

        # Validate inputs
        if not prompt.strip():
            return [], "❌ Please enter a prompt"

        generated_images = []
        generation_info_list = []

        try:
            # Apply style preset
            styled_prompt, styled_negative = self.apply_style_preset(
                style_preset, prompt, negative_prompt
            )

            # Get image size
            model_type = CONFIG["models"][model_name]["type"]
            if model_type == "sdxl" and size_name in CONFIG["image_sizes"]["SDXL"]:
                width, height = CONFIG["image_sizes"]["SDXL"][size_name]
            elif size_name in CONFIG["image_sizes"]["SD 1.5"]:
                width, height = CONFIG["image_sizes"]["SD 1.5"][size_name]
            else:
                width, height = (512, 512)

            for i in range(batch_size):
                progress((i + 0.3) / batch_size, desc=f"Generating image {i+1}/{batch_size}...")

                # Handle seed
                current_seed = seed if seed != -1 else random.randint(0, 2**32-1)
                if seed == -1 or i > 0:
                    current_seed = random.randint(0, 2**32-1)

                generator = torch.Generator(device=self.device).manual_seed(int(current_seed))

                # Record start time
                start_time = datetime.now()

                # Generate image
                with torch.no_grad():
                    result = self.pipeline(
                        prompt=styled_prompt,
                        negative_prompt=styled_negative if styled_negative.strip() else None,
                        width=width,
                        height=height,
                        num_inference_steps=int(steps),
                        guidance_scale=float(cfg_scale),
                        generator=generator
                    )
                    image = result.images[0]

                # Post-process image
                image = self.post_process_image(
                    image, brightness, contrast, saturation, sharpness
                )

                # Record generation time
                generation_time = (datetime.now() - start_time).total_seconds()
                self.generation_times.append(generation_time)

                # Save image
                filename = f"enhanced_gen_{CONFIG['generation_count']+i+1}_{current_seed}.png"
                image.save(filename)

                # Create generation info
                generation_info = {
                    "id": CONFIG["generation_count"] + i + 1,
                    "timestamp": datetime.now().isoformat(),
                    "prompt": prompt,
                    "styled_prompt": styled_prompt,
                    "negative_prompt": negative_prompt,
                    "styled_negative_prompt": styled_negative,
                    "model": model_name,
                    "size": size_name,
                    "width": width,
                    "height": height,
                    "cfg_scale": cfg_scale,
                    "steps": steps,
                    "seed": current_seed,
                    "filename": filename,
                    "style_preset": style_preset,
                    "generation_time": generation_time,
                    "post_processing": {
                        "brightness": brightness,
                        "contrast": contrast,
                        "saturation": saturation,
                        "sharpness": sharpness
                    }
                }

                self.history.append(generation_info)
                generation_info_list.append(generation_info)
                generated_images.append(image)

            CONFIG["generation_count"] += batch_size
            self.save_history()

            progress(1.0, desc="✨ Complete!")

            # Create status message
            status = f"✅ Generated {batch_size} images successfully!\n"
            status += f"⏱️ Average time: {np.mean(self.generation_times[-batch_size:]):.2f}s\n"
            status += f"📊 Total: {CONFIG['generation_count']}/{CONFIG['max_generations']}"

            return generated_images, status

        except Exception as e:
            return [], f"❌ Generation error: {str(e)}"

    def save_preset(self, name, prompt, negative_prompt, cfg_scale, steps, style_preset):
        """Save current settings as preset"""
        if not name:
            return "❌ Please enter a preset name"

        self.presets[name] = {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "cfg_scale": cfg_scale,
            "steps": steps,
            "style_preset": style_preset
        }
        self.save_presets()
        return f"✅ Preset '{name}' saved successfully!"

    def load_preset(self, name):
        """Load a saved preset"""
        if name not in self.presets:
            return None, None, 7.5, 30, "None"

        preset = self.presets[name]
        return (
            preset.get("prompt", ""),
            preset.get("negative_prompt", ""),
            preset.get("cfg_scale", 7.5),
            preset.get("steps", 30),
            preset.get("style_preset", "None")
        )

    def add_to_favorites(self, generation_id):
        """Add generation to favorites"""
        for item in self.history:
            if item["id"] == generation_id:
                if generation_id not in [f["id"] for f in self.favorites]:
                    self.favorites.append(item)
                    self.save_favorites()
                    return "⭐ Added to favorites!"
                else:
                    return "ℹ️ Already in favorites"
        return "❌ Generation not found"

    def get_generation_stats(self):
        """Get current generation statistics"""
        return {
            "total": CONFIG["generation_count"],
            "limit": CONFIG["max_generations"],
            "remaining": CONFIG["max_generations"] - CONFIG["generation_count"]
        }

    def get_statistics(self):
        """Get detailed statistics"""
        if not self.history:
            return None, None, None

        # Model usage chart
        model_counts = defaultdict(int)
        style_counts = defaultdict(int)
        daily_counts = defaultdict(int)

        for item in self.history:
            model_counts[item["model"]] += 1
            style_counts[item.get("style_preset", "None")] += 1
            date = item["timestamp"][:10]
            daily_counts[date] += 1

        # Create charts
        fig_models = go.Figure(data=[
            go.Bar(x=list(model_counts.keys()), y=list(model_counts.values()),
                   marker_color='rgba(102, 126, 234, 0.8)',
                   text=list(model_counts.values()),
                   textposition='auto')
        ])
        fig_models.update_layout(
            title="Model Usage Distribution",
            xaxis_title="Model",
            yaxis_title="Number of Generations",
            showlegend=False,
            template="plotly_dark",
            height=400
        )

        # Generation timeline
        dates = sorted(daily_counts.keys())
        values = [daily_counts[d] for d in dates]

        fig_timeline = go.Figure(data=[
            go.Scatter(x=dates, y=values, mode='lines+markers',
                      line=dict(color='rgba(78, 205, 196, 1)', width=3),
                      marker=dict(size=10, color='rgba(255, 107, 107, 0.8)'),
                      fill='tozeroy',
                      fillcolor='rgba(78, 205, 196, 0.2)')
        ])
        fig_timeline.update_layout(
            title="Generation Timeline",
            xaxis_title="Date",
            yaxis_title="Generations",
            showlegend=False,
            template="plotly_dark",
            height=400
        )

        # Style preferences pie chart
        fig_styles = go.Figure(data=[
            go.Pie(labels=list(style_counts.keys()), values=list(style_counts.values()),
                   hole=.4,
                   marker=dict(colors=px.colors.qualitative.Set3))
        ])
        fig_styles.update_layout(
            title="Style Preferences",
            template="plotly_dark",
            height=400
        )

        return fig_models, fig_timeline, fig_styles

    def export_history(self, format="json"):
        """Export generation history"""
        if format == "json":
            return json.dumps(self.history, indent=2)
        elif format == "csv":
            if not self.history:
                return "No data to export"

            df = pd.DataFrame(self.history)
            return df.to_csv(index=False)

def create_enhanced_interface():
    """Create the enhanced Gradio interface"""
    generator = EnhancedImageGenerator()

    # Custom CSS for modern UI
    css = """
    .gradio-container {
        font-family: 'Inter', 'Segoe UI', 'Roboto', sans-serif !important;
        max-width: 1600px !important;
        background: linear-gradient(135deg, #1e1e2e 0%, #2d2d44 100%);
    }
    .title-header {
        text-align: center;
        background: linear-gradient(90deg, #ff6b6b 0%, #4ecdc4 50%, #45b7d1 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        font-size: 3.5rem;
        font-weight: 800;
        margin: 30px 0;
        animation: gradient 3s ease infinite;
        background-size: 200% auto;
    }
    @keyframes gradient {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }
    .stats-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 25px;
        border-radius: 20px;
        text-align: center;
        margin: 15px 0;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        animation: pulse 2s infinite;
        position: relative;
        overflow: hidden;
    }
    .stats-card::before {
        content: '';
        position: absolute;
        top: -50%;
        left: -50%;
        width: 200%;
        height: 200%;
        background: linear-gradient(45deg, transparent, rgba(255,255,255,0.1), transparent);
        transform: rotate(45deg);
        animation: shine 3s infinite;
    }
    @keyframes shine {
        0% { transform: translateX(-100%) translateY(-100%) rotate(45deg); }
        100% { transform: translateX(100%) translateY(100%) rotate(45deg); }
    }
    @keyframes pulse {
        0% { transform: scale(1); }
        50% { transform: scale(1.02); }
        100% { transform: scale(1); }
    }
    .model-info {
        background: rgba(255,255,255,0.1);
        backdrop-filter: blur(10px);
        padding: 20px;
        border-radius: 15px;
        border: 1px solid rgba(255,255,255,0.2);
        margin: 15px 0;
        color: white;
        transition: all 0.3s ease;
    }
    .model-info:hover {
        background: rgba(255,255,255,0.15);
        transform: translateY(-2px);
        box-shadow: 0 5px 20px rgba(0,0,0,0.2);
    }
    """

    with gr.Blocks(title="🎨 Enhanced AI Image Studio", theme=gr.themes.Soft(
        primary_hue="purple",
        secondary_hue="blue",
        neutral_hue="gray",
        font=gr.themes.GoogleFont("Inter")
    ), css=css) as interface:

        # Header with animation
        gr.HTML("""
            <div class="title-header">
                🎨 Enhanced AI Image Studio
            </div>
            <div style="text-align: center; color: #a0a0a0; font-size: 1.3em; margin-bottom: 40px;">
                Professional Stable Diffusion Interface with Advanced Features
            </div>
        """)

        # Main tabs
        with gr.Tabs() as tabs:
            # Generation Tab
            with gr.Tab("🎨 Image Generation", id=0):
                # Statistics row
                with gr.Row():
                    stats_display = gr.HTML()

                with gr.Row():
                    # Left column - Advanced Controls
                    with gr.Column(scale=1):
                        with gr.Group():
                            gr.Markdown("### 🤖 Model Configuration")

                            model_selector = gr.Dropdown(
                                choices=list(CONFIG["models"].keys()),
                                value="SD 1.5 Fast",
                                label="AI Model",
                                info="Select your preferred model"
                            )

                            model_info = gr.HTML()

                            scheduler_selector = gr.Dropdown(
                                choices=CONFIG["schedulers"],
                                value="DPM Solver++",
                                label="Scheduler",
                                info="Sampling algorithm"
                            )

                            with gr.Row():
                                load_model_btn = gr.Button(
                                    "📥 Load Model",
                                    variant="primary",
                                    size="sm"
                                )

                                optimal_settings_btn = gr.Button(
                                    "⚡ Optimal Settings",
                                    variant="secondary",
                                    size="sm"
                                )

                            model_status = gr.Textbox(
                                label="Status",
                                value="❌ No model loaded",
                                interactive=False,
                                max_lines=1
                            )

                        with gr.Group():
                            gr.Markdown("### ✍️ Prompt Engineering")

                            # Preset management
                            with gr.Row():
                                preset_dropdown = gr.Dropdown(
                                    choices=list(generator.presets.keys()),
                                    label="Load Preset",
                                    scale=3
                                )
                                load_preset_btn = gr.Button("📂", scale=1)

                            prompt = gr.Textbox(
                                label="Prompt",
                                placeholder="Describe your image in detail...",
                                lines=4
                            )

                            negative_prompt = gr.Textbox(
                                label="Negative Prompt",
                                placeholder="What to avoid...",
                                lines=2,
                                value="blurry, low quality, distorted, ugly, bad anatomy"
                            )

                            with gr.Row():
                                style_preset = gr.Dropdown(
                                    choices=["None"] + list(CONFIG["style_presets"].keys()),
                                    value="None",
                                    label="Style Preset",
                                    scale=3
                                )

                                enhance_prompt_btn = gr.Button("✨", scale=1)

                        with gr.Group():
                            gr.Markdown("### ⚙️ Generation Parameters")

                            size_selector = gr.Dropdown(
                                label="Image Size",
                                choices=generator.get_available_sizes("SD 1.5 Fast"),
                                value="512x512 (Square)"
                            )

                            with gr.Row():
                                cfg_scale = gr.Slider(
                                    minimum=1.0,
                                    maximum=20.0,
                                    value=7.5,
                                    step=0.5,
                                    label="CFG Scale",
                                    info="Prompt adherence"
                                )

                                steps = gr.Slider(
                                    minimum=10,
                                    maximum=150,
                                    value=30,
                                    step=5,
                                    label="Steps",
                                    info="Quality vs Speed"
                                )

                            with gr.Row():
                                seed = gr.Number(
                                    label="Seed",
                                    value=-1,
                                    precision=0,
                                    info="-1 for random"
                                )

                                batch_size = gr.Slider(
                                    minimum=1,
                                    maximum=8,
                                    value=1,
                                    step=1,
                                    label="Batch Size"
                                )

                        with gr.Group():
                            gr.Markdown("### 🎨 Post-Processing")

                            with gr.Row():
                                brightness = gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Brightness"
                                )

                                contrast = gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Contrast"
                                )

                            with gr.Row():
                                saturation = gr.Slider(
                                    minimum=0.0,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Saturation"
                                )

                                sharpness = gr.Slider(
                                    minimum=0.0,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Sharpness"
                                )

                        # Action buttons
                        with gr.Row():
                            generate_btn = gr.Button(
                                "🚀 Generate",
                                variant="primary",
                                size="lg"
                            )

                            stop_btn = gr.Button(
                                "⏹️ Stop",
                                variant="stop",
                                size="lg"
                            )

                        with gr.Row():
                            save_preset_btn = gr.Button("💾 Save Preset", size="sm")
                            preset_name_input = gr.Textbox(
                                placeholder="Preset name...",
                                scale=2,
                                visible=False
                            )
                            random_prompt_btn = gr.Button("🎲 Random Prompt", size="sm")
                            clear_btn = gr.Button("🗑️ Clear All", size="sm")

                    # Right column - Results and Gallery
                    with gr.Column(scale=1):
                        with gr.Group():
                            gr.Markdown("### 🖼️ Generated Images")

                            gallery = gr.Gallery(
                                label="Results",
                                show_label=False,
                                elem_id="gallery",
                                columns=2,
                                rows=2,
                                object_fit="contain",
                                height=600
                            )

                            generation_status = gr.Textbox(
                                label="Generation Status",
                                interactive=False,
                                lines=3
                            )

                            with gr.Row():
                                download_all_btn = gr.Button("💾 Download All", variant="secondary")
                                add_favorite_btn = gr.Button("⭐ Add to Favorites", variant="secondary")
                                share_btn = gr.Button("🔗 Share", variant="secondary")

                        # Quick Examples
                        with gr.Group():
                            gr.Markdown("### 💡 Quick Examples")
                            gr.Examples(
                                examples=[
                                    ["A majestic dragon soaring over a mystical forest, fantasy art, highly detailed, 8k resolution, dramatic lighting", "Fantasy"],
                                    ["Portrait of a cyberpunk hacker, neon lights, futuristic city background, blade runner style, rain, reflections", "Cyberpunk"],
                                    ["Traditional Japanese garden with cherry blossoms, peaceful zen atmosphere, watercolor painting, soft colors", "Watercolor"],
                                    ["Astronaut exploring alien planet with bioluminescent plants, sci-fi concept art, volumetric lighting", "Digital Art"],
                                    ["Steampunk mechanical owl, brass gears, Victorian era, intricate details, golden hour lighting", "Photorealistic"],
                                    ["Northern lights over snowy mountains, aurora borealis, night photography, long exposure, stars", "Photorealistic"],
                                    ["Underwater coral reef, tropical fish, sun rays through water, vibrant colors, national geographic style", "Photorealistic"],
                                    ["Medieval castle on a cliff, storm approaching, dramatic sky, fantasy landscape, epic scale", "Fantasy"]
                                ],
                                inputs=[prompt, style_preset],
                                label=""
                            )

            # Analytics Tab
            with gr.Tab("📊 Analytics & History", id=1):
                with gr.Row():
                    with gr.Column():
                        refresh_stats_btn = gr.Button("🔄 Refresh Statistics", variant="primary")

                        with gr.Row():
                            model_usage_chart = gr.Plot(label="Model Usage")
                            style_pref_chart = gr.Plot(label="Style Preferences")

                        timeline_chart = gr.Plot(label="Generation Timeline")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 📜 Generation History")
                        history_display = gr.Dataframe(
                            headers=["ID", "Time", "Prompt", "Model", "Size", "Style", "Seed"],
                            label="Recent Generations",
                            interactive=False,
                            wrap=True
                        )

                        with gr.Row():
                            export_format = gr.Radio(
                                choices=["json", "csv"],
                                value="json",
                                label="Export Format"
                            )
                            export_btn = gr.Button("📥 Export History", variant="secondary")
                            export_output = gr.File(label="Download Export")

            # Help Tab
            with gr.Tab("📚 Help & Guide", id=2):
                gr.Markdown("""
                ## 🎯 Enhanced AI Image Studio User Guide

                ### 🚀 Quick Start
                1. **Select a Model**: Choose from various models based on your needs
                2. **Load the Model**: Click "Load Model" and wait for confirmation
                3. **Craft Your Prompt**: Describe your desired image in detail
                4. **Adjust Settings**: Fine-tune parameters for optimal results
                5. **Generate**: Click "Generate" and watch the magic happen!

                ### 🤖 Model Comparison
                | Model | Speed | Quality | Best For |
                |-------|-------|---------|----------|
                | SD 1.5 Fast | ⚡⚡⚡⚡⚡ | ⭐⭐⭐ | Quick iterations |
                | SD 1.5 Quality | ⚡⚡⚡ | ⭐⭐⭐⭐ | Balanced performance |
                | SDXL Base | ⚡⚡ | ⭐⭐⭐⭐⭐ | Maximum quality |
                | OpenJourney | ⚡⚡⚡⚡ | ⭐⭐⭐⭐ | Artistic styles |
                | DreamShaper | ⚡⚡⚡ | ⭐⭐⭐⭐ | Fantasy/Dreams |

                ### 🎨 Style Presets
                - **Photorealistic**: Ultra-realistic photos
                - **Digital Art**: Modern digital illustrations
                - **Anime**: Japanese animation style
                - **Fantasy**: Magical and ethereal
                - **Cyberpunk**: Futuristic neon aesthetics
                - **Oil Painting**: Traditional art style

                ### ⚙️ Parameter Guide
                - **CFG Scale (1-20)**:
                  - 1-5: Very creative, loose interpretation
                  - 7-10: Balanced creativity and accuracy
                  - 15-20: Strict prompt following

                - **Steps (10-150)**:
                  - 10-25: Fast, lower quality
                  - 30-50: Good balance
                  - 50+: Maximum quality, slower

                - **Seed**: Use same seed to reproduce exact results

                ### 💡 Pro Tips
                1. **Prompt Engineering**:
                   - Be specific and descriptive
                   - Include style keywords: "8k, highly detailed, artstation"
                   - Use negative prompts to avoid unwanted elements

                2. **Performance Optimization**:
                   - Start with fewer steps for testing
                   - Use batch generation for variations
                   - Enable memory optimizations for large images

                3. **Quality Enhancement**:
                   - Use post-processing sliders
                   - Combine style presets with custom prompts
                   - Generate at higher resolutions for details

                ### 🔧 Troubleshooting
                - **Out of Memory**: Enable CPU offload, reduce batch size
                - **Slow Generation**: Lower steps, use SD 1.5 instead of SDXL
                - **Poor Quality**: Increase steps, adjust CFG scale
                - **Model Won't Load**: Clear cache and try again
                """)

        # Helper functions
        def update_stats():
            stats = generator.get_generation_stats()
            progress_percent = (stats['total'] / stats['limit']) * 100

            return f"""
            <div class="stats-card">
                <h3>📊 Generation Progress</h3>
                <div style="background: rgba(255,255,255,0.2); border-radius: 10px; padding: 5px; margin: 10px 0;">
                    <div style="background: linear-gradient(90deg, #4ecdc4 0%, #45b7d1 {progress_percent}%, transparent {progress_percent}%);
                                height: 20px; border-radius: 10px; transition: all 0.5s ease;"></div>
                </div>
                <p style="font-size: 1.2em; margin: 10px 0;">
                    <strong>{stats['total']}</strong> / {stats['limit']} images generated
                </p>
                <p style="font-size: 1.1em; opacity: 0.9;">
                    <strong>{stats['remaining']}</strong> generations remaining
                </p>
            </div>
            """

        def update_model_info(model_name):
            info = CONFIG["models"][model_name]
            optimal = info["optimal_settings"]
            return f"""
            <div class="model-info">
                <h4>🤖 {model_name}</h4>
                <p>{info['description']}</p>
                <p><strong>Type:</strong> {info['type'].upper()}</p>
                <p><strong>Optimal Settings:</strong> {optimal['steps']} steps, {optimal['cfg']} CFG</p>
            </div>
            """

        def apply_optimal_settings(model_name):
            if model_name in CONFIG["models"]:
                optimal = CONFIG["models"][model_name]["optimal_settings"]
                return optimal["cfg"], optimal["steps"]
            return 7.5, 30

        def update_size_options(model_name):
            available_sizes = generator.get_available_sizes(model_name)
            return gr.Dropdown(
                choices=available_sizes,
                value=available_sizes[0] if available_sizes else "512x512 (Square)"
            )

        def enhance_prompt(prompt, style):
            """Add quality enhancing keywords to prompt"""
            quality_keywords = "masterpiece, best quality, highly detailed, sharp focus, professional"
            if style != "None":
                return f"{prompt}, {quality_keywords}"
            return f"{prompt}, {quality_keywords}, 8k uhd, dslr, high quality"

        def generate_random_prompt():
            """Generate a random creative prompt"""
            subjects = ["dragon", "castle", "forest", "warrior", "spaceship", "robot", "mermaid", "phoenix",
                       "wizard", "fairy", "unicorn", "samurai", "knight", "witch", "angel", "demon"]
            settings = ["mystical", "futuristic", "ancient", "underwater", "celestial", "steampunk",
                       "magical", "cyberpunk", "ethereal", "apocalyptic", "frozen", "volcanic"]
            styles = ["fantasy art", "concept art", "digital painting", "photorealistic", "oil painting",
                     "watercolor", "anime style", "3d render", "comic book style", "impressionist"]
            moods = ["epic", "serene", "dramatic", "mysterious", "peaceful", "intense", "dreamy", "dark"]

            subject = random.choice(subjects)
            setting = random.choice(settings)
            style = random.choice(styles)
            mood = random.choice(moods)

            prompts = [
                f"A {mood} {setting} {subject}, {style}, highly detailed, 8k",
                f"{subject} in a {setting} environment, {mood} atmosphere, {style}, masterpiece",
                f"Epic {subject}, {setting} atmosphere, {style}, award winning, {mood} lighting",
                f"{mood} portrait of a {subject}, {setting} background, {style}, professional",
                f"{setting} landscape with {subject}, {mood} mood, {style}, stunning composition"
            ]

            return random.choice(prompts)

        def refresh_history():
            if not generator.history:
                return []

            # Get last 100 generations
            recent = generator.history[-100:]
            history_data = []
            for item in recent:
                history_data.append([
                    item.get("id", ""),
                    item.get("timestamp", "")[:19],
                    item.get("prompt", "")[:50] + "..." if len(item.get("prompt", "")) > 50 else item.get("prompt", ""),
                    item.get("model", ""),
                    item.get("size", ""),
                    item.get("style_preset", "None"),
                    item.get("seed", "")
                ])
            return history_data[::-1]  # Reverse to show newest first

        def export_history_handler(format):
            content = generator.export_history(format)
            filename = f"generation_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.{format}"

            with open(filename, 'w', encoding='utf-8') as f:
                f.write(content)

            return filename

        # Connect all event handlers
        model_selector.change(
            fn=update_model_info,
            inputs=[model_selector],
            outputs=[model_info]
        )

        model_selector.change(
            fn=update_size_options,
            inputs=[model_selector],
            outputs=[size_selector]
        )

        load_model_btn.click(
            fn=generator.load_model,
            inputs=[model_selector, scheduler_selector],
            outputs=[model_status]
        )

        optimal_settings_btn.click(
            fn=apply_optimal_settings,
            inputs=[model_selector],
            outputs=[cfg_scale, steps]
        )

        generate_btn.click(
            fn=generator.generate_batch,
            inputs=[prompt, negative_prompt, model_selector, size_selector,
                   cfg_scale, steps, seed, batch_size, style_preset,
                   brightness, contrast, saturation, sharpness],
            outputs=[gallery, generation_status]
        ).then(
            fn=update_stats,
            outputs=[stats_display]
        ).then(
            fn=refresh_history,
            outputs=[history_display]
        )

        enhance_prompt_btn.click(
            fn=enhance_prompt,
            inputs=[prompt, style_preset],
            outputs=[prompt]
        )

        random_prompt_btn.click(
            fn=generate_random_prompt,
            outputs=[prompt]
        )

        clear_btn.click(
            fn=lambda: ("", "", -1, 1.0, 1.0, 1.0, 1.0),
            outputs=[prompt, negative_prompt, seed, brightness, contrast, saturation, sharpness]
        )

        refresh_stats_btn.click(
            fn=generator.get_statistics,
            outputs=[model_usage_chart, timeline_chart, style_pref_chart]
        )

        export_btn.click(
            fn=export_history_handler,
            inputs=[export_format],
            outputs=[export_output]
        )

        # Initialize interface
        interface.load(
            fn=update_stats,
            outputs=[stats_display]
        )

        interface.load(
            fn=update_model_info,
            inputs=[gr.State("SD 1.5 Fast")],
            outputs=[model_info]
        )

        interface.load(
            fn=refresh_history,
            outputs=[history_display]
        )

    return interface

def main():
    """Enhanced main application entry point"""
    print("🎨 Enhanced AI Image Studio")
    print("=" * 60)
    print("✨ Professional Stable Diffusion Interface")
    print("🚀 Version 2.0 with Advanced Features")
    print("=" * 60)

    # System check
    print(f"📱 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

    print(f"🔢 Generation Limit: {CONFIG['max_generations']}")
    print(f"📊 Current Count: {CONFIG['generation_count']}")
    print("=" * 60)

    # Create and launch enhanced interface
    try:
        interface = create_enhanced_interface()

        # Launch with appropriate settings
        launch_kwargs = {
            "share": True,
            "show_error": True,
            "quiet": False,
            "favicon_path": None
        }

        if not IN_COLAB:
            launch_kwargs.update({
                "server_name": "0.0.0.0",
                "server_port": 7860
            })

        print("🌐 Launching Enhanced AI Image Studio...")
        interface.launch(**launch_kwargs)

    except Exception as e:
        print(f"❌ Launch error: {e}")
        print("🔄 Attempting fallback launch...")
        interface = create_enhanced_interface()
        interface.launch(share=True)

if __name__ == "__main__":
    main()

🚀 Running in Google Colab - Enhanced Version
🎨 Enhanced AI Image Studio
✨ Professional Stable Diffusion Interface
🚀 Version 2.0 with Advanced Features
📱 Device: CUDA
🎮 GPU: Tesla T4
💾 GPU Memory: 14.7 GB
🔢 Generation Limit: 1000
📊 Current Count: 0
🌐 Launching Enhanced AI Image Studio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://70b0253fbd3304dd90.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
